Converts sqlite3 data into parquet format. 

In [1]:
import pandas as pd
import sqlite3
import sys
sys.path.insert(0, "../") # Required for database in different directory. 


In [2]:
top_or_recent = 'top'
mode = 'std'

class ScoreGen:
    def __init__(self, top_or_recent = 'top', mode = 'std', keepHD = True):
        self.conn = sqlite3.connect('../data/UserScores.db')
        self.cursor = self.conn.cursor()
        self.top_or_recent = top_or_recent
        self.mode = mode
        
        NF = 1
        HD = 8  # Removed only for no HD
        SD = 32
        NC = 512
        SO = 4096
        PF = 16384
        SV2 = 536870912
        
        if keepHD:
            self.mods_removed = NF | SD | SO | PF | SV2 | NC
        else:
            self.mods_removed = NF | SD | HD | SO | PF | SV2 | NC
            
    def __iter__(self):
        ids = self.cursor.execute(f"SELECT DISTINCT user_id FROM {self.top_or_recent}_scores_{self.mode}").fetchall()
        # ids = [(19008254, )]
        for id in ids:
            id = id[0]
            scores = self.cursor.execute(f"SELECT beatmap_id, mods FROM {self.top_or_recent}_scores_{self.mode} WHERE user_id = ? ORDER BY created_at DESC", (id,)).fetchall()
            
            to_yield = []
            for score in scores:
                bm_id, mods = score
                mods &= ~self.mods_removed
                
                to_yield.append(str(bm_id) + '-' + str(mods))
            
            yield to_yield
        
        self.conn.close()

gen = ScoreGen(top_or_recent, mode)
sentences = list(gen)

In [4]:
df = pd.DataFrame({'sentences': sentences})
# df.to_parquet(f'{top_or_recent}_sentences_{mode}.parquet', engine='pyarrow')

,sentences
0,"[1988753-0, 786021-0, 207659-0, 1608832-0, 252..."
1,"[1385212-0, 1269345-0, 1561706-0, 1535742-0, 1..."
2,"[189861-0, 189861-0, 298162-0, 298162-0, 16305..."
3,"[1675842-0, 1675842-0, 653335-0, 653335-0, 743..."
4,"[295-0, 295-0, 4566-0, 4566-0, 10462-0, 10462-..."


In [2]:
df2 = pd.read_parquet('../data/hiscore_updates.parquet', engine='pyarrow')


In [4]:
print(len(df2))
df2.head()

56995129


,user,beatmap_id,score,pp,mods,rank,score_time,update_time,mode,is_classic
0,2,75,1456312,22.7605,0,S,2007-10-06 12:14:34,2015-02-27 04:27:38,0,1
1,2,113,2783184,19.7545,0,S,2007-10-07 21:25:30,2022-10-01 11:25:16,0,1
2,2,113,2783184,17.8410,0,S,2007-10-08 05:25:30,2016-07-26 21:36:09,0,1
3,2,160,570714,15.4234,0,S,2007-10-09 02:47:16,2022-10-01 11:25:16,0,1
4,2,182,461660,20.3939,0,C,2007-10-13 03:47:01,2019-02-07 17:01:09,0,1


In [4]:
result_df = df2.groupby('user').apply(lambda x: x.nlargest(100, 'pp')).reset_index(drop=True)

: 